In [1]:
!git clone https://github.com/hiyouga/LLaMA-Factory.git

fatal: destination path 'LLaMA-Factory' already exists and is not an empty directory.


In [2]:
%cd LLaMA-Factory

/content/LLaMA-Factory


In [3]:
!pwd

/content/LLaMA-Factory


In [4]:
!pip install -r requirements.txt

In [5]:
!pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 29.1 MB/s eta 0:00:00


In [6]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-tnoi74ph
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-tnoi74ph
  Resolved https://github.com/huggingface/transformers.git to commit 919220dab1e29f4d04eacd61a197a45a4fec2613
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 39.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.48.0.dev0-py3-none-any.whl size=10328720 sha256=abb35bf20c11d772b6bc038f57a500a2df770bf0477e98c7573de199f1d316b3
  Stored in directory: /tmp/pip-ephem-wheel-cache-suhnlq42/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      S

In [7]:
!pip install -e ".[torch, metrics]"

Obtaining file:///content/LLaMA-Factory
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Using cached transformers-4.46.1-py3-none-any.whl.metadata (44 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)
Using cached transformers-4.46.1-py3-none-any.whl (10.0 MB)
  Building editable for llamafactory (pyproject.toml) ... done
  Created wheel for llamafactory: filename=llamafactory-0.9.2.dev0-0.editable-py3-none-any.whl size=23815 sha256=2cb07924c00f2e43934534d6c24f77d0241b82ce300d3d7e0fe098d61e033c7f
  Stored in directory: /tmp/pip-ephem-wheel-cache-gu235wod/wheels/de/aa/c5/27b5682c5592b7c0eecc3e208f176dedf6b11a61cf2a910b85
Successfully built llamafactory
  Attempting un

In [8]:
!pip install liger-kernel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.2/104.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 10.7 MB/s eta 0:00:00


In [9]:
import json

args = dict(
  stage="sft",
  do_train=True,
  model_name_or_path="Qwen/Qwen2-VL-7B-Instruct",
  dataset="mllm_demo,identity",
  template="qwen2_vl",
  finetuning_type="lora",
  lora_target="all",
  output_dir="qwen2vl_lora",
  per_device_train_batch_size=2,
  gradient_accumulation_steps=4,
  lr_scheduler_type="cosine",
  logging_steps=10,
  warmup_ratio=0.1,
  save_steps=1000,
  learning_rate=5e-5,
  num_train_epochs=3.0,
  max_samples=500,
  max_grad_norm=1.0,
  loraplus_lr_ratio=16.0,
  fp16=True,
  use_liger_kernel=True,
)



In [10]:
json.dump(args, open("/content/finetuningModel.json", "w", encoding="utf-8"), indent=2)

In [13]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [14]:
!llamafactory-cli train /content/finetuningModel.json

2025-01-01 19:45:13.198367: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-01 19:45:13.214748: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-01 19:45:13.235461: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-01 19:45:13.241743: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-01 19:45:13.256708: I tensorflow/core/platform/cpu_feature_guar

In [15]:
args = dict(
  model_name_or_path="Qwen/Qwen2-VL-7B-Instruct", # use official non-quantized Llama-3-8B-Instruct model
  adapter_name_or_path="qwen2vl_lora",            # load the saved LoRA adapters
  template="qwen2_vl",                     # same to the one in training
  finetuning_type="lora",                  # same to the one in training
  export_dir="qwen2vl_2b_instruct_lora_merged",              # the path to save the merged model
  export_size=2,                       # the file shard size (in GB) of the merged model
  export_device="cpu",                    # the device used in export, can be chosen from `cpu` and `cuda`
  #export_hub_model_id="your_id/your_model",         # the Hugging Face hub ID to upload model
)


In [16]:
json.dump(args, open("merge_qwen2vl.json", "w", encoding="utf-8"), indent=2)

%cd /content/LLaMA-Factory/

/content/LLaMA-Factory


In [17]:
!llamafactory-cli export merge_qwen2vl.json

2025-01-01 19:47:27.863453: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-01 19:47:27.879857: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-01 19:47:27.900721: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-01 19:47:27.907095: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-01 19:47:27.921783: I tensorflow/core/platform/cpu_feature_guar

In [18]:
final_model_path = "/content/LLaMA-Factory/qwen2vl_2b_instruct_lora_merged"

In [19]:
hf_model_repo = "Anamta98/Qwen2-VL-7B-Instruct-LoRA-FT"

In [20]:
from huggingface_hub import notebook_login

notebook_login()

In [21]:
from huggingface_hub import HfApi, HfFolder, Repository

In [22]:

# Create an instance of HfApi
api = HfApi()


In [24]:
api.upload_folder(
    folder_path=final_model_path,    # The folder containing the model files
    repo_id=hf_model_repo,                # Your authentication token
    commit_message="Initial model upload"  # Optional commit message
)

model-00002-of-00010.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

Upload 11 LFS files:   0%|          | 0/11 [00:00<?, ?it/s]

model-00001-of-00010.safetensors:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

model-00003-of-00010.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

model-00004-of-00010.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/1.86G [00:00<?, ?B/s]

model-00009-of-00010.safetensors:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

model-00010-of-00010.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Anamta98/Qwen2-VL-7B-Instruct-LoRA-FT/commit/8f955afa8210866ddf32097564447e00cad44a4f', commit_message='Initial model upload', commit_description='', oid='8f955afa8210866ddf32097564447e00cad44a4f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Anamta98/Qwen2-VL-7B-Instruct-LoRA-FT', endpoint='https://huggingface.co', repo_type='model', repo_id='Anamta98/Qwen2-VL-7B-Instruct-LoRA-FT'), pr_revision=None, pr_num=None)

In [25]:
print(f"Model pushed to: {hf_model_repo}")

Model pushed to: Anamta98/Qwen2-VL-7B-Instruct-LoRA-FT
